# 転移学習による画像分類

1. [はじめに](#はじめに)
2. [データの準備と変換](#データの準備と変換)
3. [画像のアップロード](#画像のアップロード)
4. [転移学習](#転移学習)
5. [推論](#推論)


## はじめに

このノートブックでは、画像分類のビルトインアルゴリズムを利用した転移学習を行います。転移学習とは、何らかのデータセットで学習済みのモデルを、異なるデータセットで利用可能にする方法を意味します。転移学習の一般的な方法は、学習したモデルに対して、異なるデータセットで追加学習を行う**Fine-tuning**です。転移学習は、十分な量のないデータセットを補ったり、学習時間を短縮したりするために利用されます。

ここでは、以下のような転移学習を行います。
- ビルトインアルゴリズムが用意する学習済みモデルを利用
- 学習済みモデルに対して[caltech-101データセット](http://www.vision.caltech.edu/Image_Datasets/Caltech101/)でfine-tuning

## データの準備と変換


まずは転移学習に利用するcaltech-101データセットをダウンロードします。その名前の通り、100カテゴリ＋その他カテゴリの計101カテゴリからなるデータセットです。Jupyter notebookでは先頭に`!`をつけることでシェルスクリプトを実行できますので`wget`でダウンロードして、`tar`で解凍します。解凍後のフォルダには、以下のようなファイル構造で画像が保存されています。
```
./101_ObjectCategories/(クラス名)/image_(画像ID).jpg
```
画像分類のビルトインアルゴリズムが扱うことができるファイルフォーマットは[recordio](https://mxnet.incubator.apache.org/tutorials/basic/record_io.html)と[lst](https://mxnet.incubator.apache.org/how_to/recordio.html?highlight=im2rec)です。今回はrecordio形式を利用しますので、変換のためのツール`im2rec.py`をMXNetの[githubレポジトリ](https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py)から`wget`でダウンロードします。`im2rec.py`の利用方法は以下の通りです。なおmxnetを利用しますので、右上のカーネルの表示が`conda_mxnet_p36`になっていることを確認してください。もし異なるようでしたら、丈夫のメニューKernelからChane Kernelで`conda_mxnet_p36`を選びます。

1. lstファイルを作成します。lstファイルは、`(画像のindex) (ラベルID) (画像パス)`のタブ区切りの情報です。この際に、クラスのインデックスとクラス名が出力されるのでファイル`class_index`に保存しましょう。
```
!python im2rec.py --list --recursive --train-ratio (学習データの割合) --test-ratio (テストデータの割合) (recのファイル名) (データセットのルートディレクトリ) > (クラスのインデックス情報を保存するファイル名)
```

2. lstファイルにもとづいてrecファイルを作成します。
```
!python im2rec.py --num-thread (並列処理数) caltech101 ./101_ObjectCategories
```

In [ ]:
%%time
!wget http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
!tar xvzf 101_ObjectCategories.tar.gz
!wget https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py
!python im2rec.py --list --recursive --train-ratio 0.80  --test-ratio 0.10 caltech101 ./101_ObjectCategories > class_index
!python im2rec.py --num-thread 16 --resize 200 caltech101  ./101_ObjectCategories

## 画像のアップロード


recファイルをS3にアップロードします。アップロード先は、バケット名がdefault_backet()によって自動設定されるsagemaker-{region}-{AWS account ID}で、prefixが`notebook/transfer/caltech`となります。バケット名も自由に設定できますが、世界中で唯一の名前となるような設定が必要です。



In [ ]:
import os
import urllib.request
import boto3
import re
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
bucket = sess.default_bucket()

# Upload files to S3
prefix = 'notebook/transfer/caltech'
train_input = sess.upload_data(
        path='caltech101_train.rec', 
        key_prefix=prefix)
valid_input = sess.upload_data(
        path='caltech101_val.rec', 
        key_prefix=prefix)
test_input = sess.upload_data(
        path='caltech101_test.rec', 
        key_prefix=prefix)


# Show S3 path 
print("Training data is uploaded to", train_input)
print("Validation data is uploaded to ", valid_input)
print("Test data is uploaded to ", valid_input)

## 転移学習

転移学習を以下の手順で行うことができます。
- ビルトインアルゴリズムのうち*image classification*を実行するコンテナイメージを指定します。
- コンテナイメージにもとづいて*estimator*を呼び出します。この際、学習インスタンス数やタイプ、モデルの出力パスを指定します。*image classification*では、GPUインスタンスのみで学習可能です。
- 転移学習のためのハイパーパラメータを指定します。転移学習では`use_pretrained_model=1`を指定する必要があります。ハンズオンのため最小構成（レイヤ数18、デフォルト152)で行います。
- S3のデータに対して`application/x-recordio`形式であることを明示的に指定して、`fit`を実行すると学習が始まります。



In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

img_transfer = sagemaker.estimator.Estimator(training_image,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.p2.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                   sagemaker_session=sess)
img_transfer.set_hyperparameters(
                        num_classes=101,
                        num_training_samples=7315,
                        use_pretrained_model=1,
                        num_layers = 18,
                        epochs=3,
                        learning_rate=0.01,
                        mini_batch_size=128,
                        image_shape='3,200,200',
                        top_k=2)


from sagemaker.session import s3_input
train_rec = s3_input(s3_data=train_input, content_type='application/x-recordio')
valid_rec = s3_input(s3_data=valid_input, content_type='application/x-recordio')
img_transfer.fit({'train': train_rec, 'validation': valid_rec})

## 推論

### エンドポイントの作成
学習が終わると`deploy`を呼び出すことでエンドポイントを作成することできます。


In [ ]:
img_predictor = img_transfer.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')


### テストデータに対する推論

recordio形式のファイルを作成する際に、学習に利用していないテストデータを残しています。テストデータの一覧`caltech101_test.lst`から画像をランダムに1つ選んで推論してみましょう。デフォルトではバイナリ形式で結果を出力するので、解析しやすいようjson系形式で出力するようにします。

In [ ]:
from sagemaker.predictor import  json_deserializer
img_predictor.content_type = "application/x-image"
img_predictor.deserializer = json_deserializer

import pandas as pd
import numpy as np
test_list = pd.read_table('caltech101_test.lst', header=None)
rand_index =  np.random.choice(test_list.index, 1)
file_path = './101_ObjectCategories/'+test_list.iat[int(rand_index), 2]

class_index =pd.read_csv('class_index', sep=' ',header=None, index_col=[1])

%matplotlib inline
from skimage import io, transform
from matplotlib import pyplot as plt
io.imshow(file_path)
plt.show()
with open(file_path, 'rb') as f:
    response = img_predictor.predict(f.read())
    np_resp = np.array(response)
    rank = np_resp.argsort()[::-1] # Reversing makes the array descending.
    
    # Show top 5
    for i in range(5):
        print("Class: {} (Confidence: {:.3f})".format(class_index.loc[rank[i]].values[0], np_resp[rank[i]]))


### エンドポイントの削除

In [ ]:
img_predictor.delete_endpoint()